In [ ]:
import sys
!{sys.executable} -m pip install openai-whisper
import whisper
import torch
import gradio as gr
import pandas as pd
import sqlite3
import subprocess
import os

from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
from google.colab import files
files.upload()


In [ ]:
import os
import gradio as gr
import pandas as pd
import sqlite3
import whisper
from openai import OpenAI
import re


os.environ["OPENAI_API_KEY"] = ""
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
print("✅ OpenAI client initialized.")


DATA_FOLDER = "/content/"
DB_PATH = "northwind.db"

TABLES = {
    "Employees": "employees.csv",
    "Customers": "customers.csv",
    "Orders": "orders.csv",
    "OrderDetails": "order_details.csv",
    "Products": "products.csv",
    "Categories": "categories.csv",
    "Shippers": "shippers.csv"
}

for f in TABLES.values():
    if not os.path.exists(os.path.join(DATA_FOLDER, f)):
        raise FileNotFoundError(f"❌ Missing {f}")


def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.select_dtypes(include="object").columns:
        df[col] = df[col].astype(str).str.strip()
        df[col] = df[col].str.replace(r'[\x00-\x1f]', '', regex=True)
    return df

conn = sqlite3.connect(DB_PATH)
for table, file in TABLES.items():
    df = pd.read_csv(os.path.join(DATA_FOLDER, file), encoding="latin1")
    df.columns = df.columns.str.replace(" ", "")
    df = clean_dataframe(df)
    df.to_sql(table, conn, if_exists="replace", index=False)

    print(f"✅ Loaded & cleaned {table}")
    print(f"\n--- {table} Table Preview ---")
    print(pd.read_sql(f"SELECT * FROM {table} LIMIT 10", conn))
    print("-------------------------------\n")

conn.close()


whisper_model = whisper.load_model("base")
print("✅ Whisper model loaded")

SCHEMA = """
Employees(EmployeeID, LastName, FirstName, Title)
Customers(CustomerID, CompanyName, Country)
Orders(OrderID, CustomerID, EmployeeID, OrderDate)
OrderDetails(OrderID, ProductID, UnitPrice, Quantity, Discount)
Products(ProductID, ProductName, SupplierID, CategoryID, UnitPrice)
Categories(CategoryID, CategoryName)
Shippers(ShipperID, CompanyName)
"""

# -----------------------------
# 9️⃣ Voice → SQL Logic (FIXED)
# -----------------------------
def voice_to_sql(audio_path):
    if audio_path is None:
        return "❌ No audio provided.", None

    text = whisper_model.transcribe(audio_path)["text"].strip()

    prompt = f"""
You are an expert SQLite SQL generator.

STRICT RULES:
- Use ONLY information stated in the question
- NEVER invent values or column names
- If the question mentions "sold", "sales", or "ordered":
  YOU MUST JOIN OrderDetails
- A product is SOLD only if it exists in OrderDetails
- Products table alone does NOT indicate sales
- Use DISTINCT when duplicates are possible
- Generate ONE valid SQLite SQL statement
- NO explanations
- NO markdown

Schema:
{SCHEMA}

Question:
{text}

SQL:
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=300
    )

    sql = response.choices[0].message.content.strip()
    sql = re.sub(r"```.*?\n|```", "", sql, flags=re.DOTALL).strip()

    try:
        conn = sqlite3.connect(DB_PATH)

        if sql.upper().startswith("SELECT"):
            df = pd.read_sql(sql, conn)
            conn.close()
            return f"""
🎤 Question:
{text}

🧾 SQL:
{sql}

📊 Result (first 20 rows):
{df.head(20)}
""", None

        cursor = conn.cursor()
        cursor.execute(sql)
        conn.commit()
        affected = cursor.rowcount
        conn.close()

        return f"""
🎤 Question:
{text}

🧾 SQL:
{sql}

✅ Executed successfully
Rows affected: {affected}
""", None

    except Exception as e:
        return f"""
❌ SQL ERROR

Question:
{text}

SQL:
{sql}

Error:
{e}
""", None

# -----------------------------
# 🔟 Gradio UI
# -----------------------------
iface = gr.Interface(
    fn=voice_to_sql,
    inputs=gr.Audio(type="filepath", label="🎤 Speak your database question"),
    outputs=[
        gr.Textbox(label="Result"),
        gr.File(label="⬇️ Download Updated CSV")
    ],
    title="🎓 Voice-to-SQL Assistant (Northwind)",
    description="Whisper + GPT | Correct SOLD logic using OrderDetails"
)

# -----------------------------
# 1️⃣1️⃣ Launch
# -----------------------------
iface.launch(share=True, debug=False)
